## Desafio Quarentena Dados

O desafio final das aulas de data science eh conseguir prever a nota da prova de linguagem e codigo a partir das outras notas do aluno.

O dataset fornecido contem 4 notas das outras provas de cada aluno. Sao elas:

* **NU_NOTA_CN** - Nota da Prova de Ciencias da Natureza
* **NU_NOTA_CH** - Nota da Prova de Ciencias Humanas
* **NU_NOTA_MT** - Nota da Prova de Matematica
* **NU_NOTA_REDACAO** - Nota da Prova de Redacao


Os dados fornecidos para o desafio foram divididos em 3 grupos:

* **Treino** - contendo 150.000 notas de exemplo
* **Teste**  - contendo  20.000 notas de exemplo
* **Submissao** - contendo 10.000 notas de exemplo

O erro entre a nota final e a nota que foi prevista sera calculado usando `MSE` (mean square root).

In [0]:
import pandas as pd
import numpy as np

### Carregando os dados

In [0]:
URI_TREINO    = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE     = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv?raw=true"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv?raw=true"

dados_treino  = pd.read_csv(URI_TREINO)
dados_teste   = pd.read_csv(URI_TESTE)
dados_desafio = pd.read_csv(URI_DESAFIOQT)

erro_treino  = "Erro ao carregar dados de treino"
erro_teste   = "Erro ao carregar dados de teste"
erro_desafio = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafio.shape == (10000, 5), erro_desafio

### Feature Engineering

Como são poucas as features fornecidas no dataset, eh importante pensarmos em como podemos adicionar mais informacoes no dataset que ajudem o modelo. 


#### Estatisticas

Vamos adicionar algumas informações relevantes sobre o aluno a partir das 4 provas geradas. Essas informações são:

* MIN - Nota minina tirada - Talvez ele nao tire uma nota abaixo da menor nota dele
* MAX - Nota maxima tirada - Talvez ele nao tire uma nota acima da maior nota dele
* MEDIAN - Nota mediana - A nota deve estar proxima da mediana dele
* AVERAGE - Nota media - A nota deve estar proxima da media de notas dele
* STD - Desvio padrao das notas - Diz pro modelo o quanto varia as notas dele

Importante notar que todas essas estatisticas sao calculadas sem a nota alvo. Adicionar qualquer residuo dela contaminaria a amostra. 

In [0]:
# todas as notas menos a nota alvo
features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

dados_treino['MEDIAN']  = dados_treino[features].apply(np.median, axis=1)
dados_treino['MIN']     = dados_treino[features].apply(np.min, axis=1)
dados_treino['MAX']     = dados_treino[features].apply(np.max, axis=1)
dados_treino['AVERAGE'] = dados_treino[features].apply(np.average, axis=1)
dados_treino['STD']     = dados_treino[features].apply(np.std, axis=1)

dados_teste['MEDIAN']  = dados_teste[features].apply(np.median, axis=1)
dados_teste['MIN']     = dados_teste[features].apply(np.min, axis=1)
dados_teste['MAX']     = dados_teste[features].apply(np.max, axis=1)
dados_teste['AVERAGE'] = dados_teste[features].apply(np.average, axis=1)
dados_teste['STD']     = dados_teste[features].apply(np.std, axis=1)

dados_desafio['MEDIAN']  = dados_desafio[features].apply(np.median, axis=1)
dados_desafio['MIN']     = dados_desafio[features].apply(np.min, axis=1)
dados_desafio['MAX']     = dados_desafio[features].apply(np.max, axis=1)
dados_desafio['AVERAGE'] = dados_desafio[features].apply(np.average, axis=1)
dados_desafio['STD']     = dados_desafio[features].apply(np.std, axis=1)

#### Estatisticas - Sem a nota zero

As notas zero são de alunos que nao compareceram nas provas. A falta em uma prova nao diminui a capacidade dele de tirar notas boas em outras provas, por isso deve haver uma estatistica que leva isso em conta. Para isso, vamos adicionar as mesmas estatisticas passadas, mas dessa vez desconsiderando as notas zero. 

2 Observações:

* Como a nota maxima não se altera, nao vamos adicionar ela aqui
* Caso o aluno tenha tirado todas notas zero, e removemos todas as notas zero, nao dah pra calcular as estatisticas em um vetor vazio. Nesse caso, removemos fora o aluno

In [0]:
# remove casos de 100% notas zero
dados_treino  = dados_treino.replace(0, np.NaN).dropna(how='all').fillna(0)
dados_teste   = dados_teste.replace(0, np.NaN).dropna(how='all').fillna(0)
dados_desafio = dados_desafio.replace(0, np.NaN).dropna(how='all').fillna(0)

In [0]:
dados_treino.shape, dados_teste.shape, dados_desafio.shape

((149999, 10), (20000, 10), (10000, 10))


A limpeza dos zeros removeu so um elemento do treino e deixou o teste e a submissao intactos, então não tem o que nos preocupar.

Nas linhas de baixo então geramos as novas estatisticas. Os campos vão ter os mesmo nomes dos campos anteriores, apenas acrescidos do prefixo `NZ` (non zero).

In [0]:
dados_treino_zeronan       = dados_treino[features].replace(0, np.NaN)
dados_treino['NZ_MEDIAN']  = dados_treino_zeronan[features].apply(np.nanmedian, axis=1)
dados_treino['NZ_MIN']     = dados_treino_zeronan[features].apply(np.nanmin, axis=1)
dados_treino['NZ_AVERAGE'] = dados_treino_zeronan[features].apply(np.nanmean, axis=1)
dados_treino['NZ_STD']     = dados_treino_zeronan[features].apply(np.nanstd, axis=1)

dados_teste_zeronan       = dados_teste[features].replace(0, np.NaN)
dados_teste['NZ_MEDIAN']  = dados_teste_zeronan[features].apply(np.nanmedian, axis=1)
dados_teste['NZ_MIN']     = dados_teste_zeronan[features].apply(np.nanmin, axis=1)
dados_teste['NZ_AVERAGE'] = dados_teste_zeronan[features].apply(np.nanmean, axis=1)
dados_teste['NZ_STD']     = dados_teste_zeronan[features].apply(np.nanstd, axis=1)

dados_desafio_zeronan       = dados_desafio[features].replace(0, np.NaN)
dados_desafio['NZ_MEDIAN']  = dados_desafio_zeronan[features].apply(np.nanmedian, axis=1)
dados_desafio['NZ_MIN']     = dados_desafio_zeronan[features].apply(np.nanmin, axis=1)
dados_desafio['NZ_AVERAGE'] = dados_desafio_zeronan[features].apply(np.nanmean, axis=1)
dados_desafio['NZ_STD']     = dados_desafio_zeronan[features].apply(np.nanstd, axis=1)

In [0]:
dados_treino.shape, dados_teste.shape, dados_desafio.shape

((149999, 14), (20000, 14), (10000, 14))

Agora em vez de 4, temos 13 features para treinar (14 - o alvo). Eh hora de passar tudo isso para numpy e testar os algoritmos de regressão.

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = features + ['MEDIAN', 'MIN', 'MAX', 'AVERAGE', 'STD', 'NZ_MEDIAN', 'NZ_MIN', 'NZ_AVERAGE', 'NZ_STD']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()

X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

X_desafio = dados_desafio[coluna_features].to_numpy()

Setamos uma seed para que o resultado seja sempre o mesmo, independente da execução

In [0]:
SEED = 0xCAFEF0DA

### Treino

Para o treino, iremos usar o algoritmo `GradientBoostingRegressor`. Por ser um algoritmo de arvore de decisão, não eh necessário pre processamento para que todos os dados estejam normalizados (com media em zero e desvio padrao em -1,1, como os algoritmos lineares). O loss `huber` é o mais efetivo contra outliers, que acredito ser esse o caso. 

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

np.random.seed(SEED)

params = {
    'loss': 'huber',
    'criterion': 'mse',
    'n_estimators': 173,
    'learning_rate': 0.1,
    'min_samples_split': 4,
    'min_samples_leaf': 1,
    'max_depth': 4,
    'random_state': SEED
}

model = GradientBoostingRegressor(**params)
model.fit(X_treino, Y_treino)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=0.1, loss='huber', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=4,
                          min_weight_fraction_leaf=0.0, n_estimators=173,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=3405705434, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

Depois de treinado, vamos calcular o erro na base de teste fornecida

#### Erro na validacao

In [0]:
from sklearn.metrics import mean_squared_error

predictions = model.predict(X_teste)
predictions = np.array([round(p, 1) for p in predictions])

error = mean_squared_error(Y_teste, predictions)

print ('Erro da validacao', error)

Erro da validacao 2066.7861915


#### Importancia das features

Uma coisa bacana dos modelos baseados em arvore é que eles nos fornecem a importancia de cada feature para compor a decisão final, permitindo serem mais transparentes. Abaixo vamos mostrar a importancia de cada feature nesse treino:

In [0]:
features = map(lambda x: round(x * 100, 2), model.feature_importances_)
names_features = zip(coluna_features, features)
names_features = sorted(names_features, key=lambda x:x[1], reverse=True)

for name, feature in names_features:
    print ('%15s %s %.2f' % (name, '#' * int(feature), feature))

     NU_NOTA_CH ############################################################################ 76.41
        AVERAGE ########### 11.52
     NZ_AVERAGE ###### 6.87
         MEDIAN ## 2.50
     NU_NOTA_CN  0.73
      NZ_MEDIAN  0.52
            MAX  0.49
         NZ_MIN  0.29
     NU_NOTA_MT  0.21
NU_NOTA_REDACAO  0.16
            MIN  0.11
         NZ_STD  0.11
            STD  0.08


A nota da prova de Ciencias Humanas e a media das notas correspondem a + de 90% da importancia na decisão do calculo da nota de Linguagem. 

Agora, por ultimo, a geracao final e o envio do arquivo para o desafio.

In [0]:
from google.colab import files

predictions = model.predict(X_desafio)
predictions = np.array([round(p, 1) for p in predictions])

desafio_df = pd.DataFrame(dados_desafio.ID)
desafio_df[coluna_label] = predictions

desafio_df.to_csv('PREDICAO_DESAFIOQT.csv', index=False) 
files.download('PREDICAO_DESAFIOQT.csv')